# diffusers stable-diffusion

* https://huggingface.co/stabilityai/stable-diffusion-2

* https://huggingface.co/docs/diffusers/main/en/training/dreambooth
* https://ngwaifoong92.medium.com/how-to-fine-tune-stable-diffusion-using-lora-85690292c6a8
* https://towardsdatascience.com/improving-diffusers-package-for-high-quality-image-generation-a50fff04bdd4

* https://stable-diffusion-art.com/controlnet

In [1]:
import os, glob
import random, math
import numpy as np
import pandas as pd
from PIL import Image
import torch
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
styles = pd.read_csv('stable_diffusion_artist_styles.csv')

/storage/local/stablediff/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [312]:
#model_id = "runwayml/stable-diffusion-v1-5"
model_id = "stabilityai/stable-diffusion-2-1"
#model_id = "CompViz/stable-diffusion-2"

# Use the Euler scheduler here instead
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16, safety_checker=None)
pipe = pipe.to("cuda")

Loading pipeline components...: 100%|██████████| 6/6 [00:01<00:00,  3.91it/s]


In [311]:
def prompt(prompt, n=1, style=None, path='.', negative_prompt=None, init_images=None, seed=None):
    """Prompt, assumes the correct pipe object"""

    if style != None:
        prompt += ' by %s'%style
    if negative_prompt == None:
        negative_prompt = 'disfigured, bad anatomy, low quality, ugly, tiling, poorly drawn hands, poorly drawn feet, out of frame'
    for c in range(n):
        print (prompt)
        image = pipe(prompt, negative_prompt=negative_prompt, seed=seed).images[0]
        if not os.path.exists(path):
            os.makedirs(path)
        i=1
        imgfile = os.path.join(path,prompt[:90]+'_%s.png' %i)
        while os.path.exists(imgfile):
            i+=1
            imgfile = os.path.join(path,prompt[:90]+'_%s.png' %i)
        image.save(imgfile,'png')        
    return image

In [4]:
def get_words():
    import requests
    word_site = "https://www.mit.edu/~ecprice/wordlist.10000"
    response = requests.get(word_site)
    w = response.content.splitlines()
    w = [i.decode() for i in w]
    w = [i for i in w if len(i)>3]
    return w

randwords = get_words()

In [5]:
def random_image(artist,n=1,k=3,other='',path=None):
    for i in range(n):
        words = random.choices(randwords,k=k)
        print (words)
        txt = ' '.join(words)
        if other != '':
            txt+=', '+other
        if path == None:
            path = artist
        prompt(txt, style=artist, path=path)

In [62]:
def tile_images(image_paths, outfile, grid=False, tile_width=300):
    """Make tiled image"""

    from PIL import Image, ImageDraw
    images = [Image.open(path) for path in image_paths]
      
    ratio = images[0].height / images[0].width
    tile_height = int( tile_width * ratio )
    num_rows = int(math.sqrt(len(image_paths)))
    # Calculate number of cols
    num_columns = (len(images) + num_rows - 1) // num_rows

    tiled_width = num_columns * tile_width
    tiled_height = num_rows * tile_height
    tiled_image = Image.new("RGB", (tiled_width, tiled_height))

    for idx, image in enumerate(images):      
        row = idx // num_columns
        col = idx % num_columns
        x_offset = col * tile_width
        y_offset = row * tile_height
        tiled_image.paste(image.resize((tile_width, tile_height)), (x_offset, y_offset))
    if grid == True:
        draw = ImageDraw.Draw(tiled_image)
        # Draw borders around each tile
        for row in range(num_rows):
            for col in range(num_columns):
                x1 = col * tile_width
                y1 = row * tile_height
                x2 = x1 + tile_width
                y2 = y1 + tile_height
                draw.rectangle([x1, y1, x2, y2], outline=(0, 0, 0), width=3)  

    tiled_image.save(outfile)
    return tiled_image

In [ ]:
name='mask'
media = pd.read_csv('stable_diffusion_media.csv').Name
for m in media:
    prompt(name,style=m,n=1,path=name)

In [ ]:
media

## look at varying styles

In [ ]:
random_image('studio ghibli, makoto shinkai, by greg rutkowski, intricate',n=10,path='test')

In [ ]:
prompt('angelina jolie wearing joker makeup, high quality, beautiful, highly detailed, 8k',
         style='photography',path='test',n=10)

In [ ]:
names = styles.sample(10).Name
p='julian assange'
for artist in names:
    prompt(p,style='%s expressionism' %artist,n=1,path='julian assange')
#prompt(p,style='isometric 3d',path='julian assange',n=6)

In [ ]:
#prompt('ireland',style='tolkien, robert bateman & erin hanson',path='test',n=10)
prompt('cow head logo',style='',path='test',n=20)
#prompt('alien city',style='Carl Rungius, Giger & Paul Hedley, painting',path='test',n=20)

In [ ]:
names = styles.sample(20).Name
p='happy child'
for name in names:
    prompt(p,style=name,path='test')

In [ ]:
#combinations
def combine_artists(txt, path, n=1):
    names = ['Paul Hedley','Paul Cezanne','Paul Klee','Robert Bateman',
            'Ray Harris-Ching','Erin Hanson','Carl Rungius','Inessa Garmash','Alphonse Mucha']
    for i in range(n):
        n1,n2,n3 = random.sample(names,k=3)
        prompt(txt,style='%s, %s & %s' %(n1,n2,n3),n=1,path=path)

combine_artists('taylor swift', 'test', n=10)

In [ ]:
animals = [    
    "Penguin","Giraffe","Cat","Fox",
    "Lion","Elephant","Orangutan","Chimpanzee","Pig",
    "Bear","Hippopotamus","Gorilla","Zebra",
    "Ostrich","Badger","Gazelle","Raccoon"]

for a in animals:    
    combine_artists(a, 'animals', n=2)

## random combination of a dictionary of words

In [ ]:
#artist='paul hedley'
artist='robert bateman'
#artist='albert benois'
#artist='antoine blanchard'
#artist='phillipe druillet'
#artist='moebius'
#artist='vladimir volegov'
#artist='albrecht anker'
#artist='inessa garmash'
#artist='jrr tolkien'

themes = {'tolkien':(['middle earth'],['town','mountain','forest','tower','ruins','woodland','valley'],
                     ['horses','men','dragon','lady','warrior','elves',]),
        'ireland':(['ireland'],['monastic','castle','abbey','celtic','shamrock','green','ruins','cross'],['tree','oak','birch','moon','mountain','cliffs']),
        'nature':(['pond','forest','wood','lake','coast','meadow'],['cloudy','mountain','valley','grotto'],['evening','dusk','summer','spring','autumn','winter']),
        'woman':(['woman','lady'],['african','asian','russian','italian'],['beautiful','classical','elegant']),
        'scifi':(['steampunk','futuristic'],['machine','city','vehicle','car','motorbike','train'],['retro'])}
t='woman'
for i in range(10):
    words = themes[t]
    w = [random.choice(i) for i in words]
    w += ['impressionist']
    txt = '%s_'%t +' '.join(w)
    prompt(txt, style=artist, path=artist)

## film scenes

In [ ]:
artist='paul hedley'
modf = pd.read_csv('movies.csv')
for p in modf.sample(20).name:
    prompt(p+' impressionist', path='movie scenes',style=artist)

## image-to-image

* https://blog.roboflow.com/stable-diffusion-image-to-image-pipeline/

In [2]:
from diffusers import StableDiffusionImg2ImgPipeline
model_id = "stabilityai/stable-diffusion-2-1"
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id, scheduler=scheduler, torch_dtype=torch.float16).to("cuda")

Loading pipeline components...: 100%|██████████| 6/6 [00:01<00:00,  5.07it/s]


In [3]:
def img2imgprompt(prompt, n=1, style=None, path='.', negative_prompt=None, 
                    init_images=None, strength=0.8, guidance_scale=9, seed=None):
    """Image-to-image prompt, assumes the correct pipe object"""

    if style != None:
        prompt += ' by %s'%style
    if negative_prompt == None:
        negative_prompt = 'disfigured, bad anatomy, low quality, ugly, tiling, poorly drawn hands, poorly drawn feet, out of frame'
    for c in range(n):        
        if seed == None:
            seed = torch.randint(0, 10000, (1,)).item()
        print (prompt, strength, seed)
        generator = torch.Generator(device="cuda").manual_seed(seed)
        init_images = [Image.open(image).convert("RGB").resize((768,768)) for image in init_images]
        image = pipe(prompt, negative_prompt='', image=init_images, num_inference_steps=50,
                     guidance_scale=guidance_scale, generator=generator, strength=strength).images[0]
        if not os.path.exists(path):
            os.makedirs(path)
        i=1
        imgfile = os.path.join(path,prompt[:90]+'_%02d_%d.png' %(i,seed))
        while os.path.exists(imgfile):
            i+=1
            imgfile = os.path.join(path,prompt[:90]+'_%02d_%d.png' %(i,seed))
        image.save(imgfile,'png')        
    return image

In [ ]:
names = styles.sample(10).Name
images = ['vanna2.jpg']
for n in names:
    img2imgprompt(', dramatic lighting, high quality, beautiful, highly detailed, 8k',
                  style='portrait photography',path='seeds',init_images=images, guidance_scale=7, strength=0.9)

In [ ]:
images = ['monalisa.jpg']
for s in np.arange(0.05,0.98,0.03):
    img2imgprompt('taylor swift, high quality, beautiful, highly detailed',
                  style='leonardo da vinci & paul hedley',path='monalisa6',init_images=images, strength=s, seed=187)

In [ ]:
images = ['supermanreeve.jpg']
for s in np.arange(0.05,1,0.25):
    img2imgprompt('superman',
                  style='carl rungius, paul hedley',path='superman2',init_images=images, strength=s, seed=343)

In [293]:
def vary_strength(image, text, path, seed=None):    
    for s in np.arange(0.05,0.98,0.03):
        img2imgprompt(text, path=path,init_images=[image], strength=s, seed=seed)

In [ ]:
files = glob.glob('superman2/*.png')
#x = random.sample(files, 6)
x=sorted(files[-20:])
tile_images(x, 'superman_varystrength.png', grid=True)

In [29]:
make_gif('monalisa6','monalisa6.gif')

## fine tuned models

In [ ]:
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained('/local/stablediff/', torch_dtype=torch.float16, safety_checker=None).to('cuda')
#pipe.safety_checker = lambda images, clip_input: (images, False)

In [ ]:
p=' photorealistic, high detail, high quality'
neg='cartoon, 3d, ((disfigured)), ((bad art)), ((deformed)),((extra limbs)),((close up)), weird colors, blurry, (((duplicate))), ((mutilated)), [out of frame], extra fingers, mutated hands, ((poorly drawn hands)), ((poorly drawn face)), (((mutation))), (((deformed))), ((ugly)), blurry, ((bad anatomy)), (((bad proportions))), ((extra limbs)), cloned face, (((disfigured))), out of frame, ugly, extra limbs, (bad anatomy), gross proportions, (malformed limbs), ((missing arms)), ((missing legs)), (((extra arms))), (((extra legs))), mutated hands, (fused fingers), (too many fingers), (((long neck))), Photoshop, video game, ugly, tiling, poorly drawn hands, poorly drawn feet, poorly drawn face, out of frame, mutation, mutated, extra limbs, extra legs, extra arms, disfigured, deformed, cross-eye, body out of frame, blurry, bad art, bad anatomy, 3d render'
for i in range(5):
    prompt(p,path='test',negative_prompt=neg,style='')

In [ ]:
files = glob.glob('../../twitter/julian_assange/*.png')
files = glob.glob('elvis/*.png')
x = random.sample(files, 8)
tile_images(x, 'tiled.png', grid=True)

In [28]:
def make_gif(path, outfile):
    """make gif from same condition image"""

    import glob
    from PIL import Image
    files = sorted(glob.glob(path+'/*.png'))
    #print (files)
    img = Image.open(files[0])    
    #size = img.size
    images=[]
    images = [Image.open(f) for f in files]
    images[0].save(outfile,
               save_all=True, append_images=images[1:], 
               optimize=False, duration=400, loop=0)
    return
